In [3]:
import numpy as np  
import torch  
from torch import nn    

In [4]:
# 전체 인접행렬 생성 

mat = np.array([[0,0,1,0,0,0,0],[1,0,0,0,0,0,0],[0,1,1,0,0,0,0],[1,0,1,0,0,0,0],[0,1,0,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,0,0,1],[0,0,0,0,0,1,0]])
print(mat)
print(mat.shape) 

[[0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 1 1 0 0 0 0]
 [1 0 1 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 1 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0]]
(10, 7)


### input layer랑 output layer만 있는 경우

In [5]:
#아래 코드를 좀 변형하면 될듯? 아래 코드는 hidden layer가 없어서.. 
class MaskedLinear(nn.Module):
    def __init__(self, in_dim, out_dim, indices_mask):
        """
       :param in_features: number of input features
       :param out_features: number of output features
       :param indices_mask: list of two lists containing indices for dimensions 0 and 1, used to create the mask  
       """
        super(MaskedLinear, self).__init__()    
        
 
        def backward_hook(grad): 
            # Clone due to not being allowed to modify in-place gradients
            out = grad.clone()  
            out[self.mask] = 0    
            return out 
 
        self.linear = nn.Linear(in_dim, out_dim)#.cuda()    
        self.mask = torch.ones([out_dim, in_dim]).byte()#.cuda()
        self.mask[indices_mask] = 0 # create mask
        self.linear.weight.data[self.mask] = 0 # zero out bad weights
        self.linear.weight.register_hook(backward_hook) # hook to zero out bad gradients
 
    def forward(self, input):
        return self.linear(input)
    
    


In [6]:
indices_mask = torch.ones([2,4]).byte() #[output_dim, input_dim] 
indices_mask[0,0] = 0  # [output_index, input_index]: gradient 0으로 해주고 싶은 부분 
indices_mask[1,1] = 0 
indices_mask[0,2] = 0
indices_mask[0,3] = 0
indices_mask[1,3] = 0
indices_mask

tensor([[0, 1, 0, 0],
        [1, 0, 1, 0]], dtype=torch.uint8)

In [7]:
model = MaskedLinear(4, 2, indices_mask) 

In [8]:
model.linear.weight.data

tensor([[ 0.0000,  0.2785,  0.0000,  0.0000],
        [-0.2201,  0.0000,  0.0551,  0.0000]])

### hidden layer까지 있는 경우  
- adjacency matrix의 transpose 버전인 mat_mask를 통해서 hidden layer들의 dimension을 구한다 
- 나머지 부분 바꿔서 구현

In [9]:
# 전체 인접행렬 생성 
mat = np.array([[0,0,1,0,0,0,0],[1,0,0,0,0,0,0],[0,1,1,0,0,0,0],[1,0,1,0,0,0,0],[0,1,0,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,0,0,1],[0,0,0,0,0,1,0]])
print(mat)
print(mat.shape) 

[[0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 1 1 0 0 0 0]
 [1 0 1 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 1 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0]]
(10, 7)


In [10]:
mat_mask = np.transpose(mat)
print(mat_mask)
print(mat_mask.shape)

[[0 1 0 1 0 0 0 0 0 0]
 [0 0 1 0 1 0 0 0 0 0]
 [1 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1 0]]
(7, 10)


In [11]:
mat_mask = torch.from_numpy(mat_mask)
print(mat_mask)
print(mat_mask.shape)

tensor([[0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
        [1, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]], dtype=torch.int32)
torch.Size([7, 10])


In [12]:
mat_mask[0:3, 0:5] #index (0~2, 0~4) 표현할라면 이렇게.. 

tensor([[0, 1, 0, 1, 0],
        [0, 0, 1, 0, 1],
        [1, 0, 1, 1, 0]], dtype=torch.int32)

In [13]:
mat_mask[3,0:5].sum() == 0
#이때 3이 첫번째 hidden layer의 dimension 

tensor(1, dtype=torch.uint8)

In [30]:
# hidden layer 추가한 버전
#아래 코드를 좀 변형하면 될듯? 아래 코드는 hidden layer가 없어서.. 
class MaskedLinear(nn.Module):
    def __init__(self, in_dim, out_dim, mat_mask):
        """
       :param in_features: number of input features
       :param out_features: number of output features
       :param indices_mask: list of two lists containing indices for dimensions 0 and 1, used to create the mask  
       """
        super(MaskedLinear, self).__init__()    
        
        def calculate_hidden_dim(in_dim, out_dim, mat_mask):
            #hidden dim의 정보를 담은 list를 출력한다..?
            hidden_dim_list = []
            start_col_idx = 0 
            finish_col_idx = in_dim - 1
            
            while(True):
                if finish_col_idx >= mat_mask.shape[1]:
                    break 
        
                 for i in range(start_col_idx, len(mat_mask)): 
                    if (mat_mask[i,start_col_idx:(finish_col_idx + 1)].sum() == 0):
                        hidden_dim_list += [i] 
                        start_col_idx = finish_col_idx + 1
                        finish_col_idx += i 
                        break 
            
 
        def backward_hook(grad): 
            # Clone due to not being allowed to modify in-place gradients
            out = grad.clone()  
            out[self.mask] = 0    
            return out  
        
        #이부분 바꿔야함
        self.linear1 = nn.Linear(in_dim, hidden_dim1) 
        self.mask1 = torch.ones([hidden_dim1, in_dim]).byte()
        self.mask1[mat_mask의 일부] = 0 # create mask
        self.linear.weight.data[self.mask1] = 0 # zero out bad weights
        self.linear.weight.register_hook(backward_hook) # hook to zero out bad gradients
 
    #이부분 바꿔야함
    def forward(self, input):  
        return self.linear(input) 
    
    


#### test 용 코드

In [14]:
temp = [] 
temp += [1]
temp

[1]

In [15]:
temp += [2]
temp

[1, 2]

In [16]:
len(temp)

2

In [17]:
mat_mask.shape[1]

10

In [26]:
hidden_dim_list = []
start_col_idx = 0 
finish_col_idx = 4 # 5 - 1
            
while(True):
    
    if finish_col_idx >= mat_mask.shape[1]:
        break
        
    for i in range(start_col_idx, len(mat_mask)): 
        if (mat_mask[i,start_col_idx:(finish_col_idx + 1)].sum() == 0):
            hidden_dim = i - sum(hidden_dim_list)
            hidden_dim_list += [hidden_dim] 
            start_col_idx = finish_col_idx + 1
            finish_col_idx += i 
            break

In [27]:
print(hidden_dim_list)
print(start_col_idx)
print(finish_col_idx)

[3, 2]
8
12


In [28]:
hidden_dim_list = []
sum(hidden_dim_list)

0